# MultiModality

In [1]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

### Input

In [4]:
from langchain_core.messages import HumanMessage

message = HumanMessage(
    content=[
        {
            "type": "text",
            "text": "Describe the content in this image."
        },
        {
            "type": "image_url",
            "source_url": "url",
            "url": "https://cdn.pixabay.com/photo/2025/04/10/13/31/swan-9526438_1280.jpg"
        }
    ]
)

llm.invoke([message])

BadRequestError: Error code: 400 - {'error': {'message': 'Invalid content type. image_url is only supported by certain models.', 'type': 'invalid_request_error', 'param': 'messages.[0].content.[1].type', 'code': None}}